# Ceci est un test

Je peux mettre du TS ici : 

````ts

number a = 5;

class TOTO{

const Array<T> list = [];
    
// a comment here
}

```




# Sommaire


[Motivation](#]motivation)
[Notions de Base](#notions-de-base)
[Un exemple Complet](#example)
[Notions Avancées](#advanced-notions)




## Motivations {#motivations}


 Dans cette section sera explicité la motivivation de faire du typage générique




## Notions de Base {#notions-de-bases}


 Dans cette section sera explicité les notions de base sur les types génériques (simple, double etc)




## Un exemple Complet {#example}


 Dans cette section sera explicité les notions de base sur les types génériques  (simple, double etc) dans un exemple concret avec les piles sur des étudiants, professeurs et doctorants




## Notions Avancées {#advanced-notions}


 Dans cette section sera explicité les notions avancés sur les types génériques dans un exemple concret avec une classe map, filter et reduce sur des dictionaires ! J'ai hate 


